In [77]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import copy
from sklearn.utils import shuffle

# Data Processing

Read in data to generate Pandas dataframe. If generating for the first time, will take about five minutes. Otherwise, it should take about 10 seconds.

In [78]:
datapath = '../data/Pose_Dataset/'
nFrames = 300 # number of frames per squat
df_filename = '../data/all-data.pkl' # where to save concatenated data
np_filename = '../data/np-data.npy'
name_to_label = {
    'bad_innner_thigh': 0,
    'bad_back_round': 1,
    'bad_back_warp': 2,
    'bad_head': 3,
    'bad_shallow': 4,
    'bad_toe': 5,
    'good': 6
}

#error analysis utility: for combining certain classes to see if this improves accuracy of model...if not combining any classes, pass empty matrix []
#classes_to_combine = [['bad_innner_thigh', 'bad_head']] 
#classes_to_combine = [['bad_head', 'bad_toe']] 
#classes_to_combine = [['bad_back_warp', 'bad_back_round'], ['bad_head', 'bad_toe']] 
classes_to_combine = [['bad_back_warp', 'bad_back_round']]

##have to comment out if through else opening and move the code below one tab to left if need to regenerate the pickled files.
#do the opposite to use the saved data OR just delete the files and then regenerate them

if os.path.exists(df_filename) and os.path.exists(np_filename):
    # read in pkl file
    df = pd.read_pickle(df_filename)
    with open(np_filename, 'rb') as f:
        X_train = np.load(f)
        y_train = np.load(f)
        X_val = np.load(f)
        y_val = np.load(f)
        X_test = np.load(f)
        y_test = np.load(f)
else:
    # generate pkl file and npy file

    # initialize arrays to fill in each loop iteration
    filenames = []
    datas = []
    np_datas = []
    labels = []
    
    import copy
    # read in each squat file
    for cur_dir, _, files in os.walk(datapath):
        for file in files:
            if not file.endswith('.json'):
                continue
            filename = os.path.join(cur_dir, file)
            filenames.append(filename)

            data = pd.read_json(filename).to_numpy()
            data = data[1,0:nFrames] # get data for frames. Note some files have 301 frames, truncate all to first 300
            datas.append(data)
            np_data = np.array([np.array(d) for d in data])
            if np_data.shape == (300, 171):
                np_datas.append(np_data)
                labels.append(name_to_label[cur_dir.split('/')[-2]]) #append label if appending data 
                #labels.append(name_to_label_comb[cur_dir.split('/')[-2]]) #replaced line above to check out the error analysis
        print(f"Done processing {cur_dir}")
        # move arrays into dataframe
    datas = np.array(datas) # convert to array so we can loop through
    d = {'filename': filenames}
    for i in range(nFrames):
        d[str(i)] = datas[:,i]
    df = pd.DataFrame(data=d)

    #add label column based on filename (also makes new naming for combined classes)
    new_classes = ['_or_'.join(classes) for classes in classes_to_combine]
    new_labels = []
    for i in range(len(df)):
        current_label = df['filename'][i].split('/')[3]
        for j in range(len(classes_to_combine)):
            if current_label in classes_to_combine[j]:
                current_label = new_classes[j]
        new_labels.append(current_label)
    df['label'] = new_labels  
    
    # save to pkl
    df.to_pickle(df_filename)

    # write data into numpy arrays and save
    np_datas = np.stack(np_datas)
    labels = np.array(labels)
    
    #separate data into classes 
    num_classes = len(name_to_label)
    pre_data = []
    pre_labels = []   
    for i in range(num_classes):
        indices = np.where(labels == i) #finding indices where particular class is found in labels
        pre_labels.append(labels[indices])
        pre_data.append(np_datas[indices])
        
    #choose to keep only half of the data from the 2 classes we're combining
    modified_name_to_label = copy.deepcopy(name_to_label)

    for i in range(len(classes_to_combine)):
        for j in classes_to_combine[i]:
            label = name_to_label[j]
            label_to_use = name_to_label[ classes_to_combine[i][0] ] #choosing the first label in the group of classes to combine
            modified_name_to_label[j] = label_to_use

            fraction = 1/len(classes_to_combine[i]) #if don't want to reduce data such that data in the combined class is roughly equal to data in the other non-combined classes, make fraction = 1
            pre_data[label] = pre_data[label][0:int(fraction*len( pre_data[label]))]
            pre_labels[label] = [label_to_use]*int(fraction*len( pre_labels[label])) #replacing original label with the new one based on the combined classes

    #recombine the pre-data and pre-labels
    np_datas_combined = [] 
    labels_combined = []

    for i in range(len(pre_labels)):
        np_datas_combined.extend(pre_data[i])
        labels_combined.extend(pre_labels[i])
    np_datas_combined = np.array(np_datas_combined)
    labels_combined =   np.array(labels_combined)  

    #shuffle the data
    np_datas, labels = shuffle(np_datas_combined, labels_combined) #note this is using the combined frames....not the original non-combined ones  

    #modify name_to_label and labels to account for the renumbered classes
    def reorder(name_to_label, labels ):
        #create a set from labels to get unique set of labels
        sorted_unique_labels = list( set(labels) )
        sorted_unique_labels.sort() 


        for i in range(len(sorted_unique_labels)):
            for key,value in name_to_label.items():
                if value == sorted_unique_labels[i]:
                    name_to_label[key] = i #adjust the value in the name to label dict
                    labels = np.where(labels==value, i, labels) #adjust the value in labels
        return name_to_label, labels
    
    name_to_label, labels =  reorder(modified_name_to_label, labels)
    
    
    #original splitting of train, val, test
    n = np_datas.shape[0]
    split_indices = [int(n * 0.8), int(n * 0.9)]
    train_indices, val_indices, test_indices = np.split(np.random.choice(n, n, replace=False), split_indices)

    X_train = np_datas[train_indices]
    y_train = labels[train_indices]

    X_val = np_datas[val_indices]
    y_val = labels[val_indices]

    X_test = np_datas[test_indices]
    y_test = labels[test_indices]

    with open(np_filename, 'wb') as f:
        np.save(f, X_train)
        np.save(f, y_train)
        np.save(f, X_val)
        np.save(f, y_val)
        np.save(f, X_test)
        np.save(f, y_test)    

Done processing ../data/Pose_Dataset/
Done processing ../data/Pose_Dataset/good
Done processing ../data/Pose_Dataset/good/1115_3djoints_index
Done processing ../data/Pose_Dataset/bad_toe
Done processing ../data/Pose_Dataset/bad_toe/1115_3djoints_index
Done processing ../data/Pose_Dataset/bad_shallow
Done processing ../data/Pose_Dataset/bad_shallow/1115_3djoints_index
Done processing ../data/Pose_Dataset/bad_innner_thigh
Done processing ../data/Pose_Dataset/bad_innner_thigh/1115_3djoints_index
Done processing ../data/Pose_Dataset/bad_back_round
Done processing ../data/Pose_Dataset/bad_back_round/1115_3djoints_index
Done processing ../data/Pose_Dataset/bad_back_warp
Done processing ../data/Pose_Dataset/bad_back_warp/1115_3djoints_index
Done processing ../data/Pose_Dataset/bad_head
Done processing ../data/Pose_Dataset/bad_head/1115_3djoints_index


In [79]:
print(df['label'])

0           good
1           good
2           good
3           good
4           good
          ...   
1996    bad_head
1997    bad_head
1998    bad_head
1999    bad_head
2000    bad_head
Name: label, Length: 2001, dtype: object


Print out the first five rows of the dataframe. The 'filename' column contains full filepath + name of each squat, and the remaining columns contain pose data for each of 300 frames.

In [80]:
df.head(5)

,filename,0,1,2,3,4,5,6,7,8,...,291,292,293,294,295,296,297,298,299,label
0,../data/Pose_Dataset/good/1115_3djoints_index/...,"[0.43120177353718403, 0.797377414334803, 0.844...","[0.430941564889928, 0.79869170732256, 0.847091...","[0.431044555308539, 0.799997781067416, 0.84760...","[0.431592316178638, 0.7988144430510611, 0.8469...","[0.4311132590348, 0.7942079960716251, 0.840107...","[0.43193753462167306, 0.7949407220012981, 0.84...","[0.432750845147567, 0.7839057041264701, 0.8325...","[0.432541413484046, 0.787418809894588, 0.83614...","[0.431894497467298, 0.786059366307652, 0.83555...",...,"[0.43357581090852304, 0.7943466004531391, 0.84...","[0.433069769111656, 0.7938311318961081, 0.8431...","[0.431030331629899, 0.786474443905499, 0.83585...","[0.43115121328079503, 0.7842615868752231, 0.83...","[0.43390441871996305, 0.7924671389584511, 0.84...","[0.43198091643781905, 0.7867065528136351, 0.83...","[0.43174849778878904, 0.783046079262931, 0.836...","[0.43123957336803503, 0.787923224200576, 0.840...","[0.428997741451614, 0.780915252148985, 0.83353...",good
1,../data/Pose_Dataset/good/1115_3djoints_index/...,"[0.47551463277682005, 0.9215836428119221, 0.93...","[0.47563254721491705, 0.9217684412264681, 0.93...","[0.47606479372639904, 0.921504059759201, 0.932...","[0.476369782218, 0.920659233360356, 0.93398515...","[0.474865629960791, 0.9185499246891451, 0.9351...","[0.47621913132710003, 0.9194576930028691, 0.93...","[0.47541189727106403, 0.916291807802979, 0.935...","[0.473958178841594, 0.9117857030781501, 0.9319...","[0.47338005686455703, 0.907825476525932, 0.933...",...,"[0.43516232461652304, 0.7482315863841671, 0.81...","[0.440398428175206, 0.771004741367683, 0.83682...","[0.45157124879577604, 0.815575642811675, 0.875...","[0.456352779376737, 0.843149297107819, 0.90124...","[0.464314572629178, 0.8598558365912851, 0.9082...","[0.46697763869442105, 0.8769539497506571, 0.92...","[0.46801772225950805, 0.8857252811475381, 0.92...","[0.468906910070493, 0.89422190287234, 0.928794...","[0.47246470516002803, 0.9024032699077711, 0.93...",good
2,../data/Pose_Dataset/good/1115_3djoints_index/...,"[0.487507568770135, 0.945001119666321, 0.95042...","[0.48724964719018404, 0.9447277868278361, 0.94...","[0.48733040533680005, 0.9444814068387251, 0.95...","[0.487969080175346, 0.946172353218329, 0.95415...","[0.48801193176894003, 0.946446475083851, 0.954...","[0.48742002510173404, 0.9443007012276421, 0.95...","[0.48699883703371705, 0.944366404361722, 0.957...","[0.48672516046768005, 0.9434340433389351, 0.95...","[0.48594678261619706, 0.9419660089707981, 0.95...",...,"[0.48502701306215, 0.9386721636737401, 0.95784...","[0.48510789168053, 0.9385537482653441, 0.95760...","[0.48517004926523405, 0.9378531640723361, 0.95...","[0.485991932994121, 0.9392084318376931, 0.9576...","[0.48617870299984006, 0.9400844966051861, 0.95...","[0.48613420586339506, 0.939901991724413, 0.950...","[0.48738268976887006, 0.9421225640340241, 0.95...","[0.487048790933767, 0.941068299266387, 0.94779...","[0.48686638446639, 0.9400423518104981, 0.94476...",good
3,../data/Pose_Dataset/good/1115_3djoints_index/...,"[0.47153572199932403, 0.857707360049973, 0.912...","[0.47160060745792703, 0.8573000747132821, 0.91...","[0.47036495298267605, 0.8531030652053331, 0.90...","[0.47240727772132, 0.860811007425445, 0.915781...","[0.46991840338782903, 0.8573401020290371, 0.91...","[0.47183890308743204, 0.8578533481614711, 0.91...","[0.469546441113654, 0.85047357179348, 0.904737...","[0.472174902614971, 0.854945950506319, 0.91187...","[0.46757983379684404, 0.8449749239116151, 0.90...",...,"[0.39646774404282203, 0.6211367265235841, 0.68...","[0.401195957721987, 0.6348000804749631, 0.7068...","[0.40244821179802404, 0.6407111917934291, 0.71...","[0.40574182411201004, 0.6531848616857611, 0.72...","[0.40859117574242604, 0.6741368173148691, 0.74...","[0.409277574312078, 0.6778921720769541, 0.7455...","[0.409182634792002, 0.682084981652292, 0.75031...","[0.412567976505038, 0.69516563040747, 0.759781

Print number of examples in each class

In [81]:
df_labels = df.label.unique()
for label in df_labels:
    print('%s: %i' %(label,sum(df['label']==label)))

nClasses = len(df_labels)
print('nClasses:', nClasses)

print('\n Train')
df_labels = df.label.unique()
for i in range(nClasses):
    print('%s: %i' %(i,sum(y_train==i)))

print('\n Val')
df_labels = df.label.unique()
for i in range(nClasses):
    print('%s: %i' %(i,sum(y_val==i)))

print('\n Test')
df_labels = df.label.unique()
for i in range(nClasses):
    print('%s: %i' %(i,sum(y_test==i)))

good: 293
bad_toe: 295
bad_shallow: 319
bad_innner_thigh: 230
bad_back_warp_or_bad_back_round: 592
bad_head: 272
nClasses: 6

 Train
0: 192
1: 232
2: 219
3: 252
4: 235
5: 233

 Val
0: 20
1: 30
2: 25
3: 39
4: 29
5: 27

 Test
0: 17
1: 34
2: 28
3: 28
4: 31
5: 33


In [82]:
# convert y-vals to one-hot representation # REMEMBER TO ONLY RUN THIS ONCE
y_train_onehot = tf.keras.utils.to_categorical(y_train,num_classes=nClasses)
y_val_onehot = tf.keras.utils.to_categorical(y_val,num_classes=nClasses)
y_test_onehot = tf.keras.utils.to_categorical(y_test,num_classes=nClasses)

In [83]:
BATCH_SIZE = 16
train_dset = tf.data.Dataset.from_tensor_slices((X_train,y_train_onehot)).batch(BATCH_SIZE)
val_dset = tf.data.Dataset.from_tensor_slices((X_val,y_val_onehot)).batch(BATCH_SIZE)
test_dset = tf.data.Dataset.from_tensor_slices((X_test,y_test_onehot)).batch(BATCH_SIZE)

In [84]:
# save datasets
tf.data.experimental.save(train_dset,'../data/dsets/train_dset')
tf.data.experimental.save(val_dset,'../data/dsets/val_dset')
tf.data.experimental.save(test_dset,'../data/dsets/test_dset')

In [85]:
# print element_spec for input to loading model in other notebooks
train_dset.element_spec

(TensorSpec(shape=(None, 300, 171), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 6), dtype=tf.float32, name=None))

In [86]:
##variables to use for the confusion matrices
print(df_labels,'\n', name_to_label)



['good' 'bad_toe' 'bad_shallow' 'bad_innner_thigh'
 'bad_back_warp_or_bad_back_round' 'bad_head'] 
 {'bad_innner_thigh': 0, 'bad_back_round': 1, 'bad_back_warp': 1, 'bad_head': 2, 'bad_shallow': 3, 'bad_toe': 4, 'good': 5}


display_labels = [] list is df_labels but ordering must be adjusted to go from smallest to largest  based on the values in name_to_label

ex. for df_labels = ['good' 'bad_toe' 'bad_shallow' 'bad_innner_thigh_or_bad_head'
 'bad_back_round' 'bad_back_warp']
 
 and name_to_label = {'bad_innner_thigh': 0, 'bad_back_round': 1, 'bad_back_warp': 2, 'bad_head': 0, 'bad_shallow': 3, 'bad_toe': 4, 'good': 5}
 
 display_labels = ['bad_innner_thigh_or_bad_head', 'bad_back_round', 'bad_back_warp', 'bad_shallow', 'bad_toe', 'good']

In [87]:
display_labels = ['bad_inner_thigh', 'bad_back_warp_or_bad_back_round', 'bad_head', 'bad_shallow', 'bad_toe', 'good']

